# Création du model de machine learning


#### Étude des variations des données historiques dans le but de prédire si le prix va augmenter ou diminuer


- utilisation des méthodes de classification de l'apprentissage supervisé

- calcule de la variation du prix pour chaque période de temps (chaque bougie dans notre cas)

- utilisation du calcul comme valeur cible(étiquette) dans les données d'apprentissage supervisé


#### Définition des valeurs explicatives (caractéristiques)


- Prix d'ouverture (open)

- Prix le plus haut (high)

- Prix le plus bas (low)

- Prix auquel la paire de trading a été échangée à la fin de cette bougie (close )

- Volume de transactions (volume)


#### Définition de la valeur cible (étiquette)


La valeur cible sera une variable binaire indiquant si le prix a augmenté ou diminué dans la période de temps suivante entre chaque bougie :

- 1 si le prix a augmenté.
- 0 si le prix a diminué ou est resté inchangé.


#### Prépocessing


In [27]:
import pandas as pd
import numpy as np
import warnings

warnings.filterwarnings("ignore")

In [28]:
# MongoClient pour interagir avec MongoDB
from pymongo import MongoClient

# authentification à MongoDB
# définition des informations d'identification nécessaires
# pour s'authentifier auprès de MongoDB en local
mongo_user = "admin"
mongo_password = "pass"
mongo_host = "localhost"
mongo_port = 27017


# connexion à la base de données MongoDB
client = MongoClient(
    f"mongodb://{mongo_user}:{mongo_password}@{mongo_host}:{mongo_port}/"
)

# sélection de la base de données
db = client["extract_data_binance"]

# sélection de la collection
collection = db["historical_data"]

In [29]:
# récupération des données de la collection
# et stockage dans un DataFrame
df = pd.DataFrame(list(collection.find()))

# affichage de la taille du dataframe
df.shape

(8710, 8)

In [30]:
# affichage des 5 premières lignes
df.head()

,_id,symbol,timestamp,open,high,low,close,volume
0,65fe8fd7ba05a552d62598a5,BTCUSDT,2020-03-24 10:00:00,6766.62,6806.41,6624.05,6731.00,13280.444536
1,65fe8fd7ba05a552d62598a6,BTCUSDT,2020-03-24 12:00:00,6733.16,6765.00,6543.00,6690.53,17014.878531
2,65fe8fd7ba05a552d62598a7,BTCUSDT,2020-03-24 14:00:00,6690.53,6728.15,6461.01,6587.52,16364.585652
3,65fe8fd7ba05a552d62598a8,BTCUSDT,2020-03-24 16:00:00,6587.63,6706.98,6510.01,6659.11,10638.369409
4,65fe8fd7ba05a552d62598a9,BTCUSDT,2020-03-24 18:00:00,6657.74,6739.17,6552.19,6697.39,10236.182790


In [31]:
# vérification des doublons
doublons = df.duplicated().sum()
print(f"le dataframe contient {doublons} doublons")

le dataframe contient 0 doublons


In [32]:
# vérification des valeurs manquantes
display(df.isna().sum())

_id          0
symbol       0
timestamp    0
open         0
high         0
low          0
close        0
volume       0
dtype: int64

In [33]:
# vérification des types de données
df.dtypes

_id                  object
symbol               object
timestamp    datetime64[ns]
open                float64
high                float64
low                 float64
close               float64
volume              float64
dtype: object

In [34]:
# description des statistiques quantitatives
df.describe()

,timestamp,open,high,low,close,volume
count,8710,8710.000000,8710.000000,8710.000000,8710.000000,8710.000000
mean,2020-10-24 02:56:36.647531776,22719.955141,22917.138162,22499.444618,22725.045287,6359.786263
min,2020-03-24 10:00:00,5880.500000,5944.120000,5857.760000,5881.420000,5.887034
25%,2020-05-12 00:00:00,9066.632500,9116.500000,8993.222500,9069.485000,3485.648876
50%,2020-09-22 21:00:00,11465.290000,11514.525000,11413.480000,11466.215000,5038.929672
75%,2021-03-23 11:30:00,37191.697500,37666.522500,36657.257500,37198.792500,7457.777993
max,2021-09-21 06:00:00,64509.480000,64854.000000,63915.000000,64511.210000,75186.338442
std,NaN,17679.976843,17846.744658,17488.734576,17679.802010,4832.976395


In [35]:
# calcul du nombre de modalité(valeurs différentes)pour chaque variable explicative
modalite_par_variable = df.nunique()

# affichage des modalités
print(modalite_par_variable)

_id          8710
symbol          1
timestamp    6543
open         6519
high         6208
low          6258
close        6518
volume       6543
dtype: int64


#### Calcul des variations du prix pour créer la variable cible(étiquette) dans les données d'apprentissage supervisé

- utilisation de la fonction pct_change()
- c'est une méthode de la librairie pandas
- qui permet de calculer le pourcentage de changement
- entre les valeurs successives d'une série ou d'un dataframe.
- cette fonction est couramment utilisée pour
- calculer les rendements ou les variations pour les séries temporelles financières

Pour une série temporelle ou une colonne d'un DataFrame, la fonction pct_change() calcule le pourcentage de changement entre chaque élément et son précédent.

Elle renvoie une nouvelle série ou un nouveau DataFrame où chaque élément représente le pourcentage de changement par rapport à son prédécesseur.

NaN (Not a Number) est renvoyé pour la première ligne car il n'y a pas de valeur précédente à partir de laquelle calculer le pourcentage de changement.


In [36]:
# calcul de la variation journalière
df["taux_variation"] = df["close"].pct_change() * 100


# affichage des 5 premières lignes
df.head()

,_id,symbol,timestamp,open,high,low,close,volume,taux_variation
0,65fe8fd7ba05a552d62598a5,BTCUSDT,2020-03-24 10:00:00,6766.62,6806.41,6624.05,6731.00,13280.444536,NaN
1,65fe8fd7ba05a552d62598a6,BTCUSDT,2020-03-24 12:00:00,6733.16,6765.00,6543.00,6690.53,17014.878531,-0.601248
2,65fe8fd7ba05a552d62598a7,BTCUSDT,2020-03-24 14:00:00,6690.53,6728.15,6461.01,6587.52,16364.585652,-1.539639
3,65fe8fd7ba05a552d62598a8,BTCUSDT,2020-03-24 16:00:00,6587.63,6706.98,6510.01,6659.11,10638.369409,1.086752
4,65fe8fd7ba05a552d62598a9,BTCUSDT,2020-03-24 18:00:00,6657.74,6739.17,6552.19,6697.39,10236.182790,0.574852


In [37]:
# création d'une nouvelle colonne pour la valeur cible binaire
# ajoute 1 si le prix a augmenté.
# ajoute 0 si le prix a diminué ou est resté inchangé.

df["target"] = (df["taux_variation"] > 0).astype(int)


# affichage des 5 premières lignes pour vérification
df.head()

,_id,symbol,timestamp,open,high,low,close,volume,taux_variation,target
0,65fe8fd7ba05a552d62598a5,BTCUSDT,2020-03-24 10:00:00,6766.62,6806.41,6624.05,6731.00,13280.444536,NaN,0
1,65fe8fd7ba05a552d62598a6,BTCUSDT,2020-03-24 12:00:00,6733.16,6765.00,6543.00,6690.53,17014.878531,-0.601248,0
2,65fe8fd7ba05a552d62598a7,BTCUSDT,2020-03-24 14:00:00,6690.53,6728.15,6461.01,6587.52,16364.585652,-1.539639,0
3,65fe8fd7ba05a552d62598a8,BTCUSDT,2020-03-24 16:00:00,6587.63,6706.98,6510.01,6659.11,10638.369409,1.086752,1
4,65fe8fd7ba05a552d62598a9,BTCUSDT,2020-03-24 18:00:00,6657.74,6739.17,6552.19,6697.39,10236.182790,0.574852,1


In [38]:
# vérification des valeurs manquantes
display(df.isna().sum())

_id               0
symbol            0
timestamp         0
open              0
high              0
low               0
close             0
volume            0
taux_variation    1
target            0
dtype: int64

In [39]:
# suppression des colonnes non nécessaire au machine learning
# "timestamp" car nous avons effectué le calcul de la variation journalière
# "_id" car c'est un identifiant unique
# "symbol" car nous avons un seul symbole
df = df.drop(["timestamp", "_id", "symbol"], axis=1)

# affichage pour vérification
df.head()

,open,high,low,close,volume,taux_variation,target
0,6766.62,6806.41,6624.05,6731.00,13280.444536,NaN,0
1,6733.16,6765.00,6543.00,6690.53,17014.878531,-0.601248,0
2,6690.53,6728.15,6461.01,6587.52,16364.585652,-1.539639,0
3,6587.63,6706.98,6510.01,6659.11,10638.369409,1.086752,1
4,6657.74,6739.17,6552.19,6697.39,10236.182790,0.574852,1


In [40]:
#  séparation de la variable cible des variables explicatives
feats = df.drop("target", axis=1)
target = df["target"]


# affichage pour vérification
feats.head()
target.head()

0    0
1    0
2    0
3    1
4    1
Name: target, dtype: int64

In [41]:
# importation de la fonction train_test_split
# pour séparer le jeu de données en données d'entraînement et données de test
from sklearn.model_selection import train_test_split

# séparation du jeu d'entrainement et du jeu de test
X_train, X_test, y_train, y_test = train_test_split(
    feats, target, test_size=0.20, random_state=42
)

# vérification des tailles du jeu de données
print("train Set:", X_train.shape)
print("test Set:", X_test.shape)

train Set: (6968, 6)
test Set: (1742, 6)


In [42]:
# vérification du type des variables
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8710 entries, 0 to 8709
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   open            8710 non-null   float64
 1   high            8710 non-null   float64
 2   low             8710 non-null   float64
 3   close           8710 non-null   float64
 4   volume          8710 non-null   float64
 5   taux_variation  8709 non-null   float64
 6   target          8710 non-null   int64  
dtypes: float64(6), int64(1)
memory usage: 476.5 KB


In [43]:
df.head()

,open,high,low,close,volume,taux_variation,target
0,6766.62,6806.41,6624.05,6731.00,13280.444536,NaN,0
1,6733.16,6765.00,6543.00,6690.53,17014.878531,-0.601248,0
2,6690.53,6728.15,6461.01,6587.52,16364.585652,-1.539639,0
3,6587.63,6706.98,6510.01,6659.11,10638.369409,1.086752,1
4,6657.74,6739.17,6552.19,6697.39,10236.182790,0.574852,1


In [44]:
# il n'y a pas de variables catégorielles donc pas besoin de faire de séparation entre les variables numériques et catégorielles

In [45]:
# affichage vérifications des valeurs manquantes
# dans les jeux de données d'entraînement et de test
# des variables explicatives
print("valeurs manquantes dans X_train:")
print(X_train.isna().sum())

print("valeurs manquantes dans X_test:")
print(X_test.isna().sum())

# affichage vérifications des valeurs manquantes
# dans les jeux de données d'entraînement
# et de test de la cible
print("valeurs manquantes dans y_train:")
print(y_train.isna().sum())

print("valeurs manquantes dans y_test:")
print(y_test.isna().sum())

valeurs manquantes dans X_train:
open              0
high              0
low               0
close             0
volume            0
taux_variation    0
dtype: int64
valeurs manquantes dans X_test:
open              0
high              0
low               0
close             0
volume            0
taux_variation    1
dtype: int64
valeurs manquantes dans y_train:
0
valeurs manquantes dans y_test:
0


In [46]:
# remplissage des valeurs manquantes dans la variable cible
from sklearn.impute import SimpleImputer

# création de l'imputeur pour les variables manquantes numériques avec la stratégie "median"
num_imputer = SimpleImputer(missing_values=np.nan, strategy="median")

# application de l'imputeur sur X_train et X_test
X_train_imputed = pd.DataFrame(
    num_imputer.fit_transform(X_train), columns=X_train.columns
)

X_test_imputed = pd.DataFrame(num_imputer.transform(X_test), columns=X_test.columns)

# affichage vérifications des valeurs manquantes numériques
print("valeurs manquantes numériques dans X_train:")
print(X_train_imputed.isna().sum())

print("valeurs manquantes numériques dans X_test:")
print(X_test_imputed.isna().sum())

valeurs manquantes numériques dans X_train:
open              0
high              0
low               0
close             0
volume            0
taux_variation    0
dtype: int64
valeurs manquantes numériques dans X_test:
open              0
high              0
low               0
close             0
volume            0
taux_variation    0
dtype: int64


In [47]:
# il n'y a pas d'encodage à faire car il n'y a pas de variables catégorielles

In [48]:
# assignation des variables nettoyées
X_train_clean = X_train_imputed
X_test_clean = X_test_imputed

In [49]:
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

# initialisation du modèle de régression logistique
model_rl = LogisticRegression()

# entrainement du modèle régression logistique
model_rl.fit(X_train_clean, y_train)

# initialisation du modèle arbre de décision
model_ad = DecisionTreeClassifier()

# entrainement du modèle arbre de décision
model_ad.fit(X_train_clean, y_train)

# initialisation du modèle random forest
model_rf = RandomForestClassifier()

# entrainement du modèle random forest
model_rf.fit(X_train_clean, y_train)

RandomForestClassifier()

In [50]:
# évaluation de la performance des modèles
print(
    "Accuracy score régression logistique : ",
    model_rl.score(X_test_clean, y_test),
)
print("Accuracy score arbre de décision : ", model_ad.score(X_test_clean, y_test))

print("Accuracy score random forest : ", model_rf.score(X_test_clean, y_test))

Accuracy score régression logistique :  0.9977037887485649
Accuracy score arbre de décision :  0.9994259471871412
Accuracy score random forest :  0.9994259471871412


Prédictions :

Les prédictions pour les données de test sont stockées dans la variable y_pred_test.

Les données de test sont stockées dans la variable y_test.


In [51]:
from sklearn.metrics import classification_report

# prédiction sur le jeu de test "historical_data" avec le modèle arbre de décison car c'est le plus performant
y_pred_test = model_ad.predict(X_test_clean)

In [52]:
# affichage de la matrice de confusion sur "historical_data"
display(
    pd.crosstab(
        y_test, y_pred_test, colnames=["Classes réelles"], rownames=["Classes prédites"]
    )
)

Classes réelles,0,1
Classes prédites,,
0,816,1
1,0,925


La conclusion des prédictions est la suivante :

Les prédictions ont été effectuées avec succès en utilisant le modèle de classification LogisticRegression car c'est le plus performant.

Les prédictions ont été comparées aux valeurs réelles des données de test.

Les résultats des prédictions sont stockés dans la variable y_pred_test.

Nous avons :
402 Vrais négatifs.
0 Faux négatifs.
198 Vrais positifs.
0 Faux positifs.

Ces prédictions peuvent être utilisées pour évaluer la performance des modèles de classification et prendre des décisions d'achats ou de ventes basées sur les résultats obtenus.


In [53]:
# affichage du rapport de classification sur "historical_data"
print(classification_report(y_test, y_pred_test))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00       817
           1       1.00      1.00      1.00       925

    accuracy                           1.00      1742
   macro avg       1.00      1.00      1.00      1742
weighted avg       1.00      1.00      1.00      1742



In [54]:
# définition d'une pipeline pour le prétraitement des données ou pas ?
from sklearn.pipeline import Pipeline